# Inputs
---

In [20]:
import numpy as np, zarr, nrrd
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
import tifffile

In [2]:
p = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt186/outputs/'
outdir = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/test_bigstream2'
!mkdir -p $outdir

In [3]:
fix_path = p + '/r1/stitching/export.n5'
mov_path = p + '/r7/stitching/export.n5'
exp_factor = 2  # replace this with the known expansion factor for your sample to use pre-expansion units

s1_scale = [1,2,2] # z,y,x
s3_scale = [4,8,8]
s4_scale = [9,16,16]

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr.attrs.asdict()
fix_spacing = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing * s1_scale 
fix_spacing_s3 = fix_spacing * s3_scale 
fix_spacing_s4 = fix_spacing * s4_scale 

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr.attrs.asdict()
mov_spacing = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing * s1_scale
mov_spacing_s3 = mov_spacing * s3_scale 
mov_spacing_s4 = mov_spacing * s4_scale 

# print spacings
print(fix_spacing, mov_spacing)
print(fix_spacing_s1, mov_spacing_s1)
print(fix_spacing_s3, mov_spacing_s3)
print(fix_spacing_s4, mov_spacing_s4)

# print shapes
print(fix_zarr['/c3/s0'].shape, mov_zarr['/c3/s0'].shape)
print(fix_zarr['/c3/s3'].shape, mov_zarr['/c3/s3'].shape)
print(fix_zarr['/c3/s4'].shape, mov_zarr['/c3/s4'].shape)

# write some channels
nrrd.write(outdir+'/fix.nrrd', fix_zarr['/c3/s4'][...].transpose(2,1,0), compression_level=2)
nrrd.write(outdir+'/mov.nrrd', mov_zarr['/c3/s4'][...].transpose(2,1,0), compression_level=2)

[0.21  0.115 0.115] [0.21  0.115 0.115]
[0.21 0.23 0.23] [0.21 0.23 0.23]
[0.84 0.92 0.92] [0.84 0.92 0.92]
[1.89 1.84 1.84] [1.89 1.84 1.84]
(2593, 7309, 9049) (2296, 7336, 9091)
(648, 913, 1131) (574, 917, 1136)
(288, 456, 565) (255, 458, 568)


# Alignment
---

### global

In [21]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# get global alignment channels
fix = fix_zarr['/c3/s4'][...]
mov = mov_zarr['/c3/s4'][...] #[:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

# create some masks to focus alignment only on planes below the large artifact at the top
fix_mask = np.ones(fix.shape, dtype=np.uint8)
fix_mask[:65] = 0
mov_mask = np.ones(mov.shape, dtype=np.uint8)
mov_mask[:45] = 0

# define alignment steps
ransac_kwargs = {
    'blob_sizes':[2, 8],
    'cc_radius':12,
    'match_threshold':0.4,
    'nspots':10000,
}

affine_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}

steps = [('ransac', ransac_kwargs,),
         ('affine', affine_kwargs,),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing_s4,
    mov_spacing_s4,
    steps,
    fix_mask=fix_mask,
    mov_mask=mov_mask,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s4, mov_spacing_s4,
    transform_list=[affine,],
)

# write results
np.savetxt(outdir+'/affine.mat', affine)
nrrd.write(outdir+'/affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)
tifffile.imwrite(outdir+'/affine.tiff', affine_aligned.transpose([2,1,0]))

# load precomputed results
affine = np.loadtxt(outdir+'/affine.mat')


Fewer than 50 spots found in fixed image, returning default
LEVEL:  0  ITERATION:  0  METRIC:  -0.0060492718906934055
LEVEL:  0  ITERATION:  1  METRIC:  -0.006090899593973053
LEVEL:  0  ITERATION:  2  METRIC:  -0.006073962860635501
LEVEL:  0  ITERATION:  3  METRIC:  -0.005988613324086965
LEVEL:  0  ITERATION:  4  METRIC:  -0.006176481572129269
LEVEL:  0  ITERATION:  5  METRIC:  -0.006381889463013508
LEVEL:  0  ITERATION:  6  METRIC:  -0.00652736332496646
LEVEL:  0  ITERATION:  7  METRIC:  -0.006604594577178613
LEVEL:  0  ITERATION:  8  METRIC:  -0.006613979217230575
LEVEL:  0  ITERATION:  9  METRIC:  -0.006566979052232072
LEVEL:  0  ITERATION:  10  METRIC:  -0.006466691532997087
LEVEL:  0  ITERATION:  11  METRIC:  -0.006559428235691139
LEVEL:  0  ITERATION:  12  METRIC:  -0.006747658593064638
LEVEL:  0  ITERATION:  13  METRIC:  -0.006866183983007629
LEVEL:  0  ITERATION:  14  METRIC:  -0.006912984051493983
LEVEL:  0  ITERATION:  15  METRIC:  -0.006903676642438746
LEVEL:  0  ITERATION: 

In [22]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# get global alignment channels
fix = fix_zarr['/c3/s4'][...]
mov = mov_zarr['/c3/s4'][...][:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

# create some masks to focus alignment only on planes below the large artifact at the top
fix_mask = np.ones(fix.shape, dtype=np.uint8)
fix_mask[:65] = 0
mov_mask = np.ones(mov.shape, dtype=np.uint8)
mov_mask[:45] = 0

# define alignment steps
ransac_kwargs = {
    'blob_sizes':[2, 8],
    'cc_radius':12,
    'match_threshold':0.4,
    'nspots':10000,
}

affine_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}

steps = [('ransac', ransac_kwargs,),
         ('affine', affine_kwargs,),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing_s4,
    mov_spacing_s4,
    steps,
    fix_mask=fix_mask,
    mov_mask=mov_mask,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s4, mov_spacing_s4,
    transform_list=[affine,],
)

# write results
np.savetxt(outdir+'/affine2.mat', affine)
nrrd.write(outdir+'/affine2.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)
tifffile.imwrite(outdir+'/affine2.tiff', affine_aligned.transpose([2,1,0]))

# load precomputed results
affine = np.loadtxt(outdir+'/affine2.mat')

Fewer than 50 spots found in fixed image, returning default
Optimization failed to improve metric
METRIC VALUES initial: -7.59456015081018e-05 final: -7.59456015081018e-05
Returning default
CPU times: user 4min 23s, sys: 39 s, total: 5min 3s
Wall time: 4min 28s


### local

In [15]:
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline
from bigstream.transform import apply_transform

import importlib
import bigstream
import bigstream.piecewise_align
importlib.reload(bigstream.piecewise_align)

<module 'bigstream.piecewise_align' from '/u/home/f/f7xiesnm/.conda/envs/napari/lib/python3.9/site-packages/bigstream/piecewise_align.py'>

In [16]:
%%time

# get global alignment channels
fix = fix_zarr['/c3/s3'][...]
mov = mov_zarr['/c3/s3'][...][:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

# define alignment steps
ransac_kwargs = {
    'blob_sizes':[4, 16],
    'cc_radius':16,
    'match_threshold':0.4,
    'nspots':4000,
}

affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}

steps = [('ransac', ransac_kwargs,),
         ('affine', affine_kwargs,),]

blocksize = [256, 256, 256]

cluster_kwargs = {
    'project':'scicompsoft',
    'ncpus':4,
    'threads':1,
    'min_workers':60,
    'max_workers':60,
}

# align
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_spacing_s3,
    mov_spacing_s3,
    steps,
    blocksize,
    static_transform_list=[affine,],
    cluster_kwargs=cluster_kwargs,
)

# apply affine only
deform_aligned = apply_transform(
    fix, mov,
    fix_spacing_s3, mov_spacing_s3,
    transform_list=[affine, deform],
)

# write results
nrrd.write(outdir+'/deform.nrrd', deform, compression_level=2)
nrrd.write(outdir+'/deformed.nrrd', deform_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
deform, _ = nrrd.read(outdir+'/deform.nrrd')

2023-07-24 15:52:43,613 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ekofv7z0', purging
2023-07-24 15:52:43,613 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vo0m7xn4', purging
2023-07-24 15:52:43,613 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6tw9hbpi', purging
2023-07-24 15:52:43,613 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6u_olvwk', purging
2023-07-24 15:52:43,619 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/u/home/f/f7xiesnm/.conda/envs/napari/lib/python3.9/site-packages/distributed/nanny.py", line 919, in run
    worker = worker_factory()
  File "/u/home/f/f7xiesnm/.conda/envs/napari/lib/python3.9/site-packages/distributed/worker.py", line 767, in __init__
    ServerNode.__init__(
TypeError: __init__() 

TypeError: __init__() got an unexpected keyword argument 'project'

### even more local

In [6]:
%%time

# get global alignment channels
fix = fix_zarr['/c3/s3'][...]
mov = mov_zarr['/c3/s3'][...][:, ::-1, ::-1]  # moving image data is 180 degrees rotates about z axis

deform_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(4.,),
    'control_point_spacing':200.0,
    'control_point_levels':(1,),
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.07,
        'numberOfIterations':75,
    },
    # 'metric':'C',  # correlation required too much memory, but the default MI worked well
}

steps = [('deform', deform_kwargs,),]
         
blocksize = [256, 256, 256]

cluster_kwargs = {
    'project':'scicompsoft',
    'ncpus':3,
    'threads':1,
    'min_workers':60,
    'max_workers':60,
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
    },
}

# align
deform2 = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_spacing_s3,
    mov_spacing_s3,
    steps,
    blocksize,
    static_transform_list=[affine, deform],
    cluster_kwargs=cluster_kwargs,
)

# apply affine only
deform2_aligned = apply_transform(
    fix, mov,
    fix_spacing_s3, mov_spacing_s3,
    transform_list=[affine, deform, deform2],
)
         
# write results
nrrd.write(outdir+'/deform2.nrrd', deform2, compression_level=2)
nrrd.write(outdir+'/deformed2.nrrd', deform2_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
deform2, _ = nrrd.read(outdir+'/deform2.nrrd')

NameError: name 'distributed_piecewise_alignment_pipeline' is not defined

## Histogram equalize data to visualize alignment result better
---

In [7]:
%%time

from skimage.exposure import equalize_adapthist, equalize_hist
from scipy.ndimage import gaussian_filter, grey_opening

fix_s3, _ = nrrd.read(outdir+'/fix_s3.nrrd')
deformed, _ = nrrd.read(outdir+'/deformed.nrrd')
deformed2, _ = nrrd.read(outdir+'/deformed2.nrrd')

fix_s3_corrected = equalize_hist(fix_s3, mask=(fix_s3 > 90))
fix_s3_corrected = np.round(fix_s3_corrected * 64000).astype(np.uint16)

deformed_corrected = equalize_hist(deformed, mask=(deformed > 90))
deformed_corrected = np.round(deformed_corrected * 64000).astype(np.uint16)

deformed2_corrected = equalize_hist(deformed2, mask=(deformed2 > 90))
deformed2_corrected = np.round(deformed2_corrected * 64000).astype(np.uint16)

nrrd.write(outdir+'/fix_s3_corrected.nrrd', fix_s3_corrected, compression_level=2)
nrrd.write(outdir+'/deformed_corrected.nrrd', deformed_corrected, compression_level=2)
nrrd.write(outdir+'/deformed2_corrected.nrrd', deformed2_corrected, compression_level=2)

FileNotFoundError: [Errno 2] No such file or directory: '/u/home/f/f7xiesnm/project-zipursky/easifish/results/test_bigstream2/fix_s3.nrrd'